In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import sys

pd.options.display.max_colwidth = 200
%matplotlib inline

In [2]:
data=pd.read_csv('C:/Users/Hello/Downloads/NHS/New folder/data.csv')
data.head()

,Date,Improve,Super,ImpCrit
0,01-10-2011,Get treatment I requested.,Care/ Treatment,1.0
1,01-10-2011,Provide Gym staff to get Gym up and running!,Care/ Treatment,1.0
2,01-10-2011,To look at the problems properly and give you the right medication.,Care/ Treatment,2.0
3,01-10-2011,To be more involved.,Involvement,2.0
4,01-10-2011,My drug problem was with Subutex. I was given methadone which has now made my drug dependency a lot worse. It would have been better for me to be prescribed Subutex and dealt with like that.,Care/ Treatment,2.0


In [3]:
datafinal=data.dropna(axis=0)
comment_columns = 'Improve'
target_columns = datafinal.columns[2:]
y = datafinal[target_columns]
X = datafinal[comment_columns]
y['ImpCrit'].values[y['ImpCrit'].values > 4] = 5
y['ImpCrit']=y['ImpCrit'].abs()
Y1=pd.get_dummies(y['Super'],drop_first = True)
Y2=pd.get_dummies(y['ImpCrit'],drop_first = True)
Y=pd.concat([Y1,Y2],axis=1)

C:\Users\Hello\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [4]:
import string

def clean(instr):
    return instr.translate(str.maketrans('', '', string.punctuation))
X = X.apply(clean) 
X = X.replace(r'\n',  ' ', regex=True) 

In [ ]:
embedding_index = {}
with open('../input/customerexp/glove.6B.300d.txt', encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        embedding_index[values[0]] = np.asarray(values[1:], dtype='float32')

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue

    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import  sequence
max_length = 100
embed_size = 300
batch_size = 32
epochs =100
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)

In [ ]:
train_tokenized = tokenizer.texts_to_sequences(X)
Xtrain = sequence.pad_sequences(train_tokenized, maxlen=max_length)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xtrain, Y, test_size=0.30, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN

# model definition
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, embed_size))
model_rnn.add(SimpleRNN(32))
model_rnn.add(Dense(21, activation='sigmoid'))
model_rnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model_rnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test,y_test))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN
from keras.layers import Bidirectional

# model definition
model_brnn = Sequential()
model_brnn.add(Embedding(max_features, embed_size))
model_brnn.add(Bidirectional(SimpleRNN(32)))
model_brnn.add(Dense(21, activation='sigmoid'))
model_brnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model_brnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test,y_test))

In [ ]:
from keras.layers import Dense, Embedding, Dropout, MaxPooling1D
from keras.layers import BatchNormalization, Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
model_cnn = Sequential()

model_cnn.add(Embedding(max_features, embed_size, weights=[embedding_matrix]))

model_cnn.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(3))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(BatchNormalization())

model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dropout(0.3))
model_cnn.add(Dense(21, activation='sigmoid'))

model_cnn.summary()
model_cnn.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model_cnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test,y_test))